# "Help Navigate Robots"
> "Help robots recognize the floor surface they’re standing on using data collected from Inertial Measurement Units"
- toc: false
- branch: master
- badges: true
- comments: true
- categories: [fastpages, jupyter]
- image: images/some_folder/your_image.png
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix

In [12]:
df = pd.read_csv("X_train.csv")
df2 = pd.read_csv("y_train.csv")

In [13]:
df.head()

,row_id,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z
0,0_0,0,0,-0.75853,-0.63435,-0.10488,-0.10597,0.107650,0.017561,0.000767,-0.74857,2.1030,-9.7532
1,0_1,0,1,-0.75853,-0.63434,-0.10490,-0.10600,0.067851,0.029939,0.003385,0.33995,1.5064,-9.4128
2,0_2,0,2,-0.75853,-0.63435,-0.10492,-0.10597,0.007275,0.028934,-0.005978,-0.26429,1.5922,-8.7267
3,0_3,0,3,-0.75852,-0.63436,-0.10495,-0.10597,-0.013053,0.019448,-0.008974,0.42684,1.0993,-10.0960
4,0_4,0,4,-0.75852,-0.63435,-0.10495,-0.10596,0.005135,0.007652,0.005245,-0.50969,1.4689,-10.4410


In [14]:
df2.head()

,series_id,group_id,surface
0,0,13,fine_concrete
1,1,31,concrete
2,2,20,concrete
3,3,31,concrete
4,4,22,soft_tiles


In [15]:
df2.shape

(3810, 3)

## Feature Engineering

In [16]:
df['orientation_T'] = (df['orientation_X']**2 + df['orientation_Y']**2 +
                               df['orientation_Z']**2)**0.5
df['linear_acceleration_T'] = (df['linear_acceleration_X']**2 + df['linear_acceleration_Y']**2 +
                               df['linear_acceleration_Z']**2)**0.5
df['angular_velocity_T'] = (df['angular_velocity_X']**2 + df['angular_velocity_Y']**2 +
                               df['angular_velocity_Z']**2)**0.5

In [17]:
df_agg = df.groupby(['series_id']).agg({'orientation_X': ['mean', min, max,sum],
                                       'orientation_Y': ['mean', min ,max,sum],
                                       'orientation_Z': ['mean', min ,max,sum],
                                        'orientation_W': ['mean', min ,max,sum],
                                        'orientation_T': ['mean', min ,max,sum],
                                       'linear_acceleration_X': ['mean', min ,max,sum],
                                       'linear_acceleration_Y': ['mean', min ,max,sum],
                                       'linear_acceleration_Z': ['mean', min ,max,sum],
                                        'linear_acceleration_T': ['mean', min ,max,sum],
                                       'angular_velocity_X': ['mean', min ,max,sum],
                                       'angular_velocity_Y': ['mean', min ,max,sum],
                                       'angular_velocity_Z': ['mean', min ,max,sum],
                                       'angular_velocity_T': ['mean', min ,max,sum]})

In [18]:
list(df_agg)

[('orientation_X', 'mean'),
 ('orientation_X', 'min'),
 ('orientation_X', 'max'),
 ('orientation_X', 'sum'),
 ('orientation_Y', 'mean'),
 ('orientation_Y', 'min'),
 ('orientation_Y', 'max'),
 ('orientation_Y', 'sum'),
 ('orientation_Z', 'mean'),
 ('orientation_Z', 'min'),
 ('orientation_Z', 'max'),
 ('orientation_Z', 'sum'),
 ('orientation_W', 'mean'),
 ('orientation_W', 'min'),
 ('orientation_W', 'max'),
 ('orientation_W', 'sum'),
 ('orientation_T', 'mean'),
 ('orientation_T', 'min'),
 ('orientation_T', 'max'),
 ('orientation_T', 'sum'),
 ('linear_acceleration_X', 'mean'),
 ('linear_acceleration_X', 'min'),
 ('linear_acceleration_X', 'max'),
 ('linear_acceleration_X', 'sum'),
 ('linear_acceleration_Y', 'mean'),
 ('linear_acceleration_Y', 'min'),
 ('linear_acceleration_Y', 'max'),
 ('linear_acceleration_Y', 'sum'),
 ('linear_acceleration_Z', 'mean'),
 ('linear_acceleration_Z', 'min'),
 ('linear_acceleration_Z', 'max'),
 ('linear_acceleration_Z', 'sum'),
 ('linear_acceleration_T', 'mean

In [19]:
df_agg[('orientation_X','range')] = df_agg[('orientation_X', 'max')] - df_agg[('orientation_X', 'min')]
df_agg[('orientation_Y','range')] = df_agg[('orientation_Y', 'max')] - df_agg[('orientation_Y', 'min')]
df_agg[('orientation_Z','range')] = df_agg[('orientation_Z', 'max')] - df_agg[('orientation_Z', 'min')]
df_agg[('orientation_W','range')] = df_agg[('orientation_W', 'max')] - df_agg[('orientation_W', 'min')]
df_agg[('orientation_T','range')] = df_agg[('orientation_T', 'max')] - df_agg[('orientation_T', 'min')]
df_agg[('linear_acceleration_X','range')] = df_agg[('linear_acceleration_X', 'max')] - df_agg[('linear_acceleration_X', 'min')]
df_agg[('linear_acceleration_Y','range')] = df_agg[('linear_acceleration_Y', 'max')] - df_agg[('linear_acceleration_Y', 'min')]
df_agg[('linear_acceleration_Z','range')] = df_agg[('linear_acceleration_Z', 'max')] - df_agg[('linear_acceleration_Z', 'min')]
df_agg[('linear_acceleration_T','range')] = df_agg[('linear_acceleration_T', 'max')] - df_agg[('linear_acceleration_T', 'min')]
df_agg[('angular_velocity_X','range')] = df_agg[('angular_velocity_X', 'max')] - df_agg[('angular_velocity_X', 'min')]
df_agg[('angular_velocity_Y','range')] = df_agg[('angular_velocity_Y', 'max')] - df_agg[('angular_velocity_Y', 'min')]
df_agg[('angular_velocity_Z','range')] = df_agg[('angular_velocity_Z', 'max')] - df_agg[('angular_velocity_Z', 'min')]
df_agg[('angular_velocity_T','range')] = df_agg[('angular_velocity_T', 'max')] - df_agg[('angular_velocity_T', 'min')]

In [20]:
df_agg.head()

orientation_X                              orientation_Y           \
                   mean      min      max        sum          mean      min   
series_id                                                                     
0             -0.758666 -0.75953 -0.75822  -97.10922     -0.634008 -0.63456   
1             -0.958606 -0.95896 -0.95837 -122.70162      0.241867  0.24074   
2             -0.512057 -0.51434 -0.50944  -65.54334     -0.846171 -0.84779   
3             -0.939169 -0.93968 -0.93884 -120.21364      0.310140  0.30943   
4             -0.891301 -0.89689 -0.88673 -114.08647      0.428144  0.41646   

                              orientation_Z                  ...          \
               max        sum          mean       min        ...           
series_id                                                    ...           
0         -0.63306  -81.15298     -0.105474 -0.106140        ...           
1          0.24270   30.95897      0.031650  0.030504        ...           
2         -0.84490 -108.30988     -0.129371 -0.130300        ...           
3          0.31147   39.69794      0.038955  0.037922        ...           
4          0.43740   54.80246      0.060056  0.058247        ...           

          orientation_W orientation_T linear_acceleration_X  \
                  range         range                 range   
series_id                                                     
0              0.001460      0.000160               4.71820   
1              0.002220      0.000331               8.29360   
2              0.001157      0.000078               4.44630   
3              0.005030      0.000717               7.99660   
4              0.001940      0.000272               2.09851   

          linear_acceleration_Y linear_acceleration_Z linear_acceleration_T  \
                          range                 range                 range   
series_id                                                                     
0                      5.310983                6.2439              5.458424   
1                      8.834200               14.1831             11.118339   
2                      7.464500                6.7548              5.931175   
3                     17.568100               19.2859             18.769065   
4                      4.476030                3.5260              3.413774   

          angular_velocity_X angular_velocity_Y angular_velocity_Z  \
                       range              range              range   
series_id                                                            
0                   0.268060           0.152102           0.081901   
1                   0.538220           0.246410           0.250760   
2                   0.294630           0.199756           0.104427   
3                   0.920650           0.303930           0.158759   
4                   0.184974           0.075533           0.150568   

          angular_velocity_T  
                       range  
series_id                     
0                   0.151781  
1                   0.267779  
2                   0.161048  
3                   0.509122  
4                   0.140159  

[5 rows x 65 columns]

In [21]:
df_agg.shape

(3810, 65)

In [22]:
le = LabelEncoder()

In [23]:
le.fit(df2['surface'])

LabelEncoder()

In [24]:
df2['surface']= le.transform(df2['surface'])

In [25]:
df2.head()

,series_id,group_id,surface
0,0,13,2
1,1,31,1
2,2,20,1
3,3,31,1
4,4,22,6


## Modelling

In [26]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(df_agg, df2['surface'], test_size=0.3) # 70% training and 30% test

In [31]:
clf = RandomForestClassifier(n_estimators=100,n_jobs=-1,min_samples_leaf = 3)
clf.fit(X_train,y_train)
clf_predictions = clf.predict(X_test) 
clf.score(X_test,y_test)

0.8442694663167104

In [32]:
clf.score(X_train,y_train)

0.9835020622422197

In [33]:
feature_importances = pd.DataFrame(clf.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
feature_importances

importance
orientation_X         min      0.037489
                      max      0.036277
                      mean     0.035278
orientation_W         mean     0.028306
orientation_T         mean     0.027860
linear_acceleration_X range    0.027695
orientation_X         sum      0.027230
orientation_W         sum      0.026694
                      min      0.026412
orientation_Z         mean     0.025123
                      min      0.024066
orientation_W         max      0.023581
orientation_T         sum      0.023560
linear_acceleration_Z range    0.022880
angular_velocity_Y    range    0.022406
orientation_T         max      0.022220
linear_acceleration_T range    0.022149
orientation_Z         sum      0.021620
linear_acceleration_T max      0.021409
orientation_Z         max      0.019985
orientation_T         min      0.019738
linear_acceleration_T mean     0.019334
orientation_Y         mean     0.018758
                      max      0.018637
                      min      0.017842
linear_acceleration_Y range    0.017529
orientation_Y         sum      0.017040
linear_acceleration_Z min      0.016843
linear_acceleration_T sum      0.015739
linear_acceleration_X max      0.015262
...                                 ...
angular_velocity_Z    min      0.011951
angular_velocity_T    range    0.011903
linear_acceleration_Z max      0.011790
angular_velocity_T    max      0.011645
angular_velocity_X    max      0.010919
                      min      0.010749
linear_acceleration_Y max      0.010397
angular_velocity_Y    mean     0.009591
angular_velocity_Z    sum      0.009376
angular_velocity_T    sum      0.009289
angular_velocity_Z    mean     0.008967
linear_acceleration_T min      0.008915
orientation_X         range    0.008802
orientation_T         range    0.008663
angular_velocity_Z    max      0.007859
orientation_Y         range    0.007482
angular_velocity_T    min      0.007371
angular_velocity_Y    sum      0.007285
angular_velocity_Z    range    0.006595
orientation_W         range    0.006150
linear_acceleration_Y sum      0.006062
                      mean     0.005781
orientation_Z         range    0.005657
angular_velocity_Y    min      0.005649
linear_acceleration_Z sum      0.005229
                      mean     0.004893
linear_acceleration_X sum      0.004389
angular_velocity_X    mean     0.003930
linear_acceleration_X mean     0.003908
angular_velocity_X    sum      0.003907

[65 rows x 1 columns]

In [34]:
cm = confusion_matrix(y_test, clf_predictions)

In [35]:
print(cm)

[[ 39   4   0   0   0   2   3   1   3]
 [  4 194   3   0   6   2   3   7   7]
 [  0  11  88   0   1   5   3   3   6]
 [  0   0   0   4   0   0   0   0   1]
 [  4   4   0   0  78   1   0   0   6]
 [  2   7   1   0   1 211   4   2   5]
 [  1   4   2   0   0   6  74   0   4]
 [  1   9   1   0   1   4   2 130   7]
 [  0   5   3   0   0   8   2   6 147]]


In [54]:
le.classes_

array(['carpet', 'concrete', 'fine_concrete', 'hard_tiles',
       'hard_tiles_large_space', 'soft_pvc', 'soft_tiles', 'tiled',
       'wood'], dtype=object)

## Score the test dataset and create the submission file

In [70]:
sub = pd.read_csv('X_test.csv')

In [71]:
sub.head()

,row_id,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z
0,0_0,0,0,-0.025773,-0.98864,-0.14801,0.003350,-0.006524,-0.001071,-0.027390,0.10043,4.2061,-5.5439
1,0_1,0,1,-0.025683,-0.98862,-0.14816,0.003439,-0.113960,0.083987,-0.060590,-0.70889,3.9905,-8.0273
2,0_2,0,2,-0.025617,-0.98861,-0.14826,0.003571,-0.080518,0.114860,-0.037177,1.45710,2.2828,-11.2990
3,0_3,0,3,-0.025566,-0.98862,-0.14817,0.003609,0.070067,0.033820,-0.035904,0.71096,1.8582,-12.2270
4,0_4,0,4,-0.025548,-0.98866,-0.14792,0.003477,0.152050,-0.029016,-0.015314,3.39960,2.7881,-10.4100


In [87]:
df_agg_test = sub.groupby(['series_id']).agg({'orientation_X': ['mean', min ,max],
                                       'orientation_Y': ['mean', min ,max],
                                       'orientation_Z': ['mean', min ,max],
                                        'orientation_W': ['mean', min ,max],
                                       'linear_acceleration_X': ['mean', min ,max],
                                       'linear_acceleration_Y': ['mean', min ,max],
                                       'linear_acceleration_Z': ['mean', min ,max],
                                       'angular_velocity_X': ['mean', min ,max],
                                       'angular_velocity_Y': ['mean', min ,max],
                                       'angular_velocity_Z': ['mean', min ,max]})

In [88]:
df_agg_test.head()

orientation_X                     orientation_Y                    \
                   mean       min       max          mean      min      max   
series_id                                                                     
0             -0.025810 -0.026418 -0.025156     -0.988644 -0.98873 -0.98854   
1             -0.932288 -0.933720 -0.931480      0.330271  0.32661  0.33227   
2             -0.230186 -0.231410 -0.227130      0.961448  0.96109  0.96217   
3              0.164661  0.163320  0.167500      0.975293  0.97485  0.97551   
4             -0.253600 -0.269380 -0.236370      0.955712  0.95150  0.96018   

          orientation_Z                     orientation_W    ...     \
                   mean       min       max          mean    ...      
series_id                                                    ...      
0             -0.148006 -0.148720 -0.147480      0.003147    ...      
1              0.043416  0.042283  0.044053     -0.140968    ...      
2              0.144340  0.143560  0.145110     -0.042394    ...      
3              0.146153  0.145820  0.146490      0.018096    ...      
4              0.142326  0.141120  0.143060     -0.044067    ...      

          linear_acceleration_Z angular_velocity_X                      \
                            max               mean       min       max   
series_id                                                                
0                       -3.9960           0.000994 -0.225610  0.231270   
1                       -3.6473           0.010602 -0.131980  0.284850   
2                       -2.1986          -0.003484 -0.302820  0.250240   
3                       -7.8266           0.004221 -0.046763  0.066896   
4                       -8.9277           0.005279 -0.029431  0.052902   

          angular_velocity_Y                     angular_velocity_Z            \
                        mean       min       max               mean       min   
series_id                                                                       
0                   0.002629 -0.137970  0.114860          -0.002971 -0.102590   
1                   0.014625 -0.041890  0.084099          -0.036900 -0.105560   
2                   0.009079 -0.161160  0.154590          -0.026607 -0.283060   
3                  -0.006815 -0.027883  0.015618           0.026272 -0.030321   
4                   0.069537  0.054452  0.091660          -0.211458 -0.240100   

                     
                max  
series_id            
0          0.097635  
1          0.037424  
2          0.138020  
3          0.081661  
4         -0.164500  

[5 rows x 30 columns]

In [89]:
df_agg_test['prediction']=clf.predict(df_agg_test)

In [90]:
df_agg_test.head()

orientation_X                     orientation_Y                    \
                   mean       min       max          mean      min      max   
series_id                                                                     
0             -0.025810 -0.026418 -0.025156     -0.988644 -0.98873 -0.98854   
1             -0.932288 -0.933720 -0.931480      0.330271  0.32661  0.33227   
2             -0.230186 -0.231410 -0.227130      0.961448  0.96109  0.96217   
3              0.164661  0.163320  0.167500      0.975293  0.97485  0.97551   
4             -0.253600 -0.269380 -0.236370      0.955712  0.95150  0.96018   

          orientation_Z                     orientation_W    ...      \
                   mean       min       max          mean    ...       
series_id                                                    ...       
0             -0.148006 -0.148720 -0.147480      0.003147    ...       
1              0.043416  0.042283  0.044053     -0.140968    ...       
2              0.144340  0.143560  0.145110     -0.042394    ...       
3              0.146153  0.145820  0.146490      0.018096    ...       
4              0.142326  0.141120  0.143060     -0.044067    ...       

          angular_velocity_X                     angular_velocity_Y            \
                        mean       min       max               mean       min   
series_id                                                                       
0                   0.000994 -0.225610  0.231270           0.002629 -0.137970   
1                   0.010602 -0.131980  0.284850           0.014625 -0.041890   
2                  -0.003484 -0.302820  0.250240           0.009079 -0.161160   
3                   0.004221 -0.046763  0.066896          -0.006815 -0.027883   
4                   0.005279 -0.029431  0.052902           0.069537  0.054452   

                    angular_velocity_Z                     prediction  
                max               mean       min       max             
series_id                                                              
0          0.114860          -0.002971 -0.102590  0.097635          4  
1          0.084099          -0.036900 -0.105560  0.037424          1  
2          0.154590          -0.026607 -0.283060  0.138020          7  
3          0.015618           0.026272 -0.030321  0.081661          6  
4          0.091660          -0.211458 -0.240100 -0.164500          6  

[5 rows x 31 columns]

In [91]:
df_agg_test['surface']= le.inverse_transform(df_agg_test['prediction'])

C:\Users\UI774014\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [99]:
df_agg_test.to_csv('submission.csv')